# Software Undergound Rendezvous: Feb 6 2021
## Importance of noise estimates when solving inverse problems in the "real" world
## Part 1 Generate results for analysis
### Sean Walker

----

## Import modules


In [1]:
%matplotlib inline

In [2]:
from pathlib import Path
import pickle

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import colors

from scipy.stats import gmean

from discretize import TreeMesh
from discretize.utils import mkvc, refine_tree_xyz, extract_core_mesh

from SimPEG.utils import surface2ind_topo
from SimPEG import (
    maps,
    data,
    data_misfit,
    regularization,
    optimization,
    inverse_problem,
    inversion,
    directives,
    utils,
)
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG.electromagnetics.static.utils.static_utils import (
    apparent_resistivity,
    source_receiver_midpoints
)

from scripts.dc_inv_utils import *

try:
    from pymatsolver.direct import Pardiso as Solver
    print('Using Paradiso Solver')
except ImportError:
    print('Using LU Solver')
    from SimPEG import SolverLU as Solver


Using Paradiso Solver
Using Paradiso Solver


In [3]:
# take this out after done editting
%load_ext autoreload
%autoreload 2


**Note** when using the Paradiso solver you may get many warnings that make it hard to read the inversion output. You need at turn off warnings by setting a variable.

On mac\/\*nx: export KMP_WARNINGS=0  (can be inlcuded in a profile file)

On windows: set KMP_WARNINGS=0 (can be set as an environment variable)

----

## Demonstrate using DC Resistivity data

### SimPEG 2.5D DC Resistivity Least-Squares Inversion

Here we invert a line of DC resistivity data to recover an electrical conductivity model. 
We formulate the corresponding inverse problems as least-squares optimization problems. For this tutorial, I am going to ignore pretty much all of specifics of how the invesion is set up and runs except the for data misfit.

Short intro to DC Resistivity data is here:

![Image](https://gpg.geosci.xyz/_images/Pseudo_PDP_East.gif)

You can find background on [DC Resistivity](https://gpg.geosci.xyz/content/DC_resistivity/index.html) data at [Geophysics for Practicing Geoscientists](https://gpg.geosci.xyz). The image is from the GPG website.



Short intro to Least-Squares Inversion:



You can find a [SimPEG](https://simpeg.xyz/) DC Resisivity inversion tutorial [here](https://docs.simpeg.xyz/content/tutorials/05-dcr/plot_inv_2_dcr2d.html#sphx-glr-content-tutorials-05-dcr-plot-inv-2-dcr2d-py)

## Load Data and define Survey

Here we load the observed data and define the DC and IP survey geometry.




In [4]:
# load data

# path to the directory containing our data
data_path = Path('data')

# files to work with
topo_filename = Path(data_path / "rv_dipole_dipole_topo.txt")
dc_data_filename = Path(data_path / "rv_dipole_dipole_dc.txt")

# load files
topo_xz = np.loadtxt(topo_filename)
dobs_dc = np.loadtxt(dc_data_filename)

# Extract source and receiver electrode locations and the observed data
#  we are assuming the dc and ip tx-rx pairs are the same. In this case they are.
A_electrodes = np.c_[dobs_dc[:, 0], np.zeros(dobs_dc.shape[0])]
B_electrodes = np.c_[dobs_dc[:, 1], np.zeros(dobs_dc.shape[0])]
M_electrodes = np.c_[dobs_dc[:, 2], np.zeros(dobs_dc.shape[0])]
N_electrodes = np.c_[dobs_dc[:, 3], np.zeros(dobs_dc.shape[0])]
dobs_dc = dobs_dc[:, -1]

# Define survey
unique_tx, k = np.unique(np.c_[A_electrodes, B_electrodes], axis=0, return_index=True)
n_sources = len(k)
k = np.r_[k, len(A_electrodes) + 1]

source_list = []
for ii in range(0, n_sources):

    # MN electrode locations for receivers. Each is an (N, 3) numpy array
    M_locations = M_electrodes[k[ii] : k[ii + 1], :]
    N_locations = N_electrodes[k[ii] : k[ii + 1], :]
    receiver_list = [dc.receivers.Dipole(M_locations, N_locations, data_type="volt")]

    # AB electrode locations for source. Each is a (1, 3) numpy array
    A_location = A_electrodes[k[ii], :]
    B_location = B_electrodes[k[ii], :]
    source_list.append(dc.sources.Dipole(receiver_list, A_location, B_location))

# Define survey
dc_survey = dc.survey.Survey(source_list)

# Define the a data object. Uncertainties are added later
dc_data = data.Data(dc_survey, dobs=dobs_dc)

# Calc app res and midx and mad mid_z used in plotting pseudo sections
app_res = apparent_resistivity(dc_data)
app_res_mean = gmean(app_res)
mid_x, mid_z = source_receiver_midpoints(dc_survey)

We now have a `Data` object and a `Survey` object for the DC resistivity data. We also calculated apparent resistivity and mid points/pseudo depths from the array to plot the data.

Now set up the mesh and define the active part of the model

In [5]:
# Mesh
# all thes values were determined by looking at the data set
#  there are probably 
core_x_min = 0
core_x_max = 1900
core_x_width = core_x_max - core_x_min
core_z_min = -600
core_z_max = 0
core_z_width = core_z_max - core_z_min

dh = 12.5  # base cell width 25m / 4
x_min = -500.  # domain width x
x_max = 2400.
dom_width_x = x_max - x_min
z_max = 0.
z_min = -1100.
dom_width_z = z_max - z_min
# Calculate num. base cells x and z must be powers of 2 of base cell
#  for tree mesh
nbcx = 2 ** int(np.ceil(np.log(dom_width_x / dh) / np.log(2.0)))
nbcz = 2 ** int(np.ceil(np.log(dom_width_z / dh) / np.log(2.0)))
# calculate the actual size of the mesh from on number of base cells.
actual_x_width = nbcx*dh
actual_z_width = nbcz*dh
# define the origin
x_origin = core_x_min - (actual_x_width - core_x_width)/2
z_origin = -1*actual_z_width
# Define the base mesh
hx = [(dh, nbcx)]
hz = [(dh, nbcz)]
mesh = TreeMesh([hx, hz], x0=[x_origin, z_origin])
# Mesh refinement based on topography
mesh = refine_tree_xyz(
    mesh, topo_xz, octree_levels=[1], method="surface", finalize=False
)

# Mesh refinement near transmitters and receivers
electrode_locations = np.r_[
    dc_survey.locations_a,
    dc_survey.locations_b,
    dc_survey.locations_m,
    dc_survey.locations_n,
]

unique_locations = np.unique(electrode_locations, axis=0)

mesh = refine_tree_xyz(
    mesh, unique_locations, octree_levels=[2, 4], method="radial",
    finalize=False)

# Refine core mesh region
xp, zp = np.meshgrid([core_x_min, core_x_max], [core_z_min, core_z_max])
xyz = np.c_[mkvc(xp), mkvc(zp)]
mesh = refine_tree_xyz(mesh, xyz, octree_levels=[0, 2, 2], method="box",
                       finalize=False)

mesh.finalize()

# Find cells that lie below surface topography
ind_active = surface2ind_topo(mesh, topo_xz)

# Shift electrodes to the surface of discretized topography
dc_survey.drape_electrodes_on_topography(mesh, ind_active, option="top")

# Define conductivity model in S/m (or resistivity model in Ohm m)
air_conductivity = np.log(1e-8)
# bacground value we will use the mean apparent resistivity
background_conductivity = np.log(1/app_res_mean)

active_map = maps.InjectActiveCells(mesh, ind_active, np.exp(air_conductivity))
nC = int(ind_active.sum())

conductivity_map = active_map * maps.ExpMap()

# Define model
starting_conductivity_model = background_conductivity * np.ones(nC)

plotting_map = maps.ActiveCells(mesh, ind_active, np.nan)


In [6]:
# mesh core
core_defn=[core_x_min, core_x_max, core_z_min, core_z_max]
# Electrode locs (re-calc)
electrode_locations = np.r_[
    dc_survey.locations_a,
    dc_survey.locations_b,
    dc_survey.locations_m,
    dc_survey.locations_n,
]

unique_locations = np.unique(electrode_locations, axis=0)

## Store results and other files for use in discussion

In [12]:
# path to the directory containing our results
results_path = Path('results')

In [13]:
# Pickle stuff we need for setting up the problem and running inversions
#  start by putting it in a dict
out_fields = ['mesh', 'dc_survey', 'dc_data', 'starting_conductivity_model', 'plotting_map', 'conductivity_map', 'ind_active',
                'core_defn', 'unique_locations', 'app_res_mean', 'app_res', 'mid_x', 'mid_z']
out_values = [mesh, dc_survey, dc_data, starting_conductivity_model, plotting_map, conductivity_map, ind_active,
                core_defn, unique_locations, app_res_mean, app_res, mid_x, mid_z]
out_dict = dict(zip(out_fields, out_values))

# setup file name
setup_filename = Path(results_path / "setup.pkl")

save_results_dict(out_dict, setup_filename)

## Assign Uncertainties

Inversion with SimPEG requires that we define standard deviation on our data.
This represents our estimate of the noise in our data.

These values are used to form the data weighting matrix Wdii = 1/stdii

We need to choose a value.

5% is fairly common starting point. But why not 10% or 1%?

Should the large offsets have larger errors (n = 4,5: 15%  and n = 6: 20%?).

Should the error estimate have a minimum value sometimes called a noise floor? We choose this as an actual noise floor: 2 * noise level? Or a samll number to prevent 1/std being too large 1e-6?


## Run a bunch of inversions to see what happens

Plot phi_d and phi_m from the various inversions.

Choose an optimal beta based on the curve flattening out.

Show that while phi_d is different for each of these the model is the same.

Good reminder of why choosing an arbitrary error estimate and ftiing to your theoretical target misfit is a bad idea.

### Test #1: Change the percentage

Run inversion with 5 and 10%

Plot errors as Voltages in pseudo section and histogram 

Based on the phi_d equation we should be able see that the relative errors are the same for each (1/data amplitude). The difference is a constant (1/percentage). So the shape of the phi_d and phi_m curves should be the same. The amplitudes and beta values will be different.



In [14]:
# Compute standard deviations
std_dc_5_per = 0.05 * np.abs(dc_data.dobs)
# run inversion
out_5_per, out_all_5_per = setup_and_run_std_inv(mesh, dc_survey, dc_data, std_dc_5_per,
                                                 conductivity_map, ind_active,
                                                 starting_conductivity_model)



        SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
        ***Done using same Solver and solverOpts as the problem***
model has any nan: 0
=============================== Projected GNCG ===============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
x0 has any nan: 0
   0  1.39e+05  3.67e+04  0.00e+00  3.67e+04    7.10e+03      0              
   1  1.39e+04  1.66e+04  4.94e-02  1.73e+04    2.74e+03      0              
   2  1.39e+03  8.20e+03  3.14e-01  8.64e+03    1.16e+03      0   Skip BFGS  
   3  1.39e+02  4.62e+03  1.43e+00  4.82e+03    5.93e+02      0   Skip BFGS  
   4  1.39e+01  2.63e+03  6.10e+00  2.72e+03    2.60e+02      0   Skip BFGS  
   5  1.39e+00  1.79e+03  1.49e+01  1.81e+03    1.89e+02      0   Skip BFGS  
   6  1.39e-01  1.36e+03  2.06e+01  1.37e+03    2.01e+02      0              
   7  1.39e-02  1.02e+03  2.90e+01

In [35]:
# Pickle stuff we need for setting up the problem and running inversions
#  start by putting it in a dict
out_fields = ['phi_d', 'phi_m', 'beta', 'inv_out', 'std_dc']
out_values = [out_5_per.phi_d, out_5_per.phi_m, out_5_per.beta, out_all_5_per.outDict, std_dc_5_per]
out_dict = dict(zip(out_fields, out_values))

# setup file name
inv_std_dc_5_per_filename = Path(results_path / "inv_std_dc_5_per.pkl")

save_results_dict(out_dict, inv_std_dc_5_per_filename)

In [ ]:
it = 2
m_it = out_all_5_per.outDict[it]['m']
dpred_it = out_all_5_per.outDict[it]['dpred']
phid_it = (dc_data.dobs - dpred_it)/std_dc_5_per
fig = plt.figure(figsize=(16, 8))
gs = GridSpec(2, 4, figure=fig)
ax = fig.add_subplot(gs[0, :-1])
mod = mesh.plot_image(
    plotting_map * np.log10(np.exp(m_it)),
    grid=False,
    clim=(-3, -1),
    ax=ax,
    pcolorOpts={"cmap": "viridis"},
)
ax.plot(unique_locations[:,0], unique_locations[:,1], 'ko')
ax.set_xlim(core_x_min, core_x_max)
ax.set_ylim(core_z_min, core_z_max)
ax.set_title(f"Recovered model iteration {it}")
fig.colorbar(mod[0], ax=ax, orientation='vertical', label='log10 conductivity')
ax2 = fig.add_subplot(gs[1, :-1])
d_phid = ax2.scatter(mid_x, mid_z, c=phid_it)
c_phid = fig.colorbar(d_phid, ax=ax2, orientation='vertical')
ax2.set_xlim(core_x_min, core_x_max)
ax2.set_ylim(core_z_min, core_z_max)
ax2.axis('equal')
ax2.set_title(f"Data misfit {it}")
ax2.set_xlabel('X')
ax2.set_ylabel('a (50m) x n')
ax3 = fig.add_subplot(gs[1,-1])
h_dobs = ax3.hist(phid_it, bins=25)
ax3.set_title('$\phi_d$')
plt.show()

In [ ]:
# Compute standard deviations
std_dc_10_per = 0.10 * np.abs(dc_data.dobs)
# run inversion
out_10_per, out_all_10_per = setup_and_run_std_inv(mesh, dc_survey, dc_data, std_dc_10_per,
                                                 conductivity_map, ind_active,
                                                 starting_conductivity_model)

In [ ]:
num_it = len(out_10_per.phi_d)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.semilogy(np.linspace(1, num_it, num_it), out_10_per.phi_d,'ko-')
ax2.semilogy(np.linspace(1, num_it, num_it), out_10_per.phi_m,'ko-')

In [ ]:
it = 2
m_it = out_all_10_per.outDict[it]['m']
dpred_it = out_all_10_per.outDict[it]['dpred']
phid_it = (dc_data.dobs - dpred_it)/std_dc_10_per
fig = plt.figure(figsize=(16, 8))
gs = GridSpec(2, 4, figure=fig)
ax = fig.add_subplot(gs[0, :-1])
mod = mesh.plot_image(
    plotting_map * np.log10(np.exp(m_it)),
    grid=False,
    clim=(-3, -1),
    ax=ax,
    pcolorOpts={"cmap": "viridis"},
)
ax.plot(unique_locations[:,0], unique_locations[:,1], 'ko')
ax.set_xlim(core_x_min, core_x_max)
ax.set_ylim(core_z_min, core_z_max)
ax.set_title(f"Recovered model iteration {it}")
fig.colorbar(mod[0], ax=ax, orientation='vertical', label='log10 conductivity')
ax2 = fig.add_subplot(gs[1, :-1])
d_phid = ax2.scatter(mid_x, mid_z, c=phid_it)
c_phid = fig.colorbar(d_phid, ax=ax2, orientation='vertical')
ax2.set_xlim(core_x_min, core_x_max)
ax2.set_ylim(core_z_min, core_z_max)
ax2.axis('equal')
ax2.set_title(f"Data misfit {it}")
ax2.set_xlabel('X')
ax2.set_ylabel('a (50m) x n')
ax3 = fig.add_subplot(gs[1,-1])
h_dobs = ax3.hist(phid_it, bins=25)
ax3.set_title('$\phi_d$')
plt.show()

### Test #2: Variable percentage

Run inversion n <= 3: 10%, 3 < n <= 5: 15%, n > 5: 20%
Run inversion n <= 3: 5%, 3 < n <= 4: 10%, n > 4: 15%

Plot errors as Voltages in pseudo section and histogram 

Errors are different so we would expect inversion to behave differently.

In [ ]:
# Calculate the n-values
n_val = mid_z/50
# Compute standard deviations
std_var_per_1 = np.zeros(len(n_val))
std_var_per_1[abs(n_val) <= 3] = 0.1
std_var_per_1[(abs(n_val) > 3) & (abs(n_val) <= 5)] = 0.15
std_var_per_1[abs(n_val) > 5] = 0.2
std_var_per_1 *= np.abs(dc_data.dobs)
# run inversion
out_var_per_1, out_all_var_per_1 = setup_and_run_std_inv(mesh, dc_survey, dc_data, std_var_per_1,
                                                 conductivity_map, ind_active,
                                                 starting_conductivity_model)

In [ ]:
# Compute standard deviations
std_var_per_2 = np.zeros(len(n_val))
std_var_per_2[abs(n_val) <= 3] = 0.05
std_var_per_2[(abs(n_val) > 3) & (abs(n_val) <= 4)] = 0.10
std_var_per_2[abs(n_val) > 4] = 0.15
std_var_per_2 *= dc_data.dobs
# run inversion
out_var_per_2, out_all_var_per_2 = setup_and_run_std_inv(mesh, dc_survey, dc_data, std_var_per_2,
                                                 conductivity_map, ind_active,
                                                 starting_conductivity_model)

### Test #3: % and floor

Run inversion 5% + 5% of std dev
Run inversion 5% + 2 * smallest value

Plot errors as Voltages in pseudo section and histogram 

Errors are different so we would expect inversion to behave differently.

In [ ]:
%matplotlib inline
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)
 
# generate x values
x = np.linspace(0, 2 * np.pi, 100)
 
 
def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))
 
 
@interact(x, w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(x, w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
#    if len(ax.lines) > 1:
#        [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')

In [ ]:
display(slid_num)

In [ ]:
slid_num.value